In [1]:
"""
Combine no-prior all-sky background scan with with Prior

Save those max TS

02_get_background_TS_with_healpix.py
"""

'\nCombine no-prior all-sky background scan with with Prior\n\nSave those max TS\n\n02_get_background_TS_with_healpix.py\n'

In [2]:
import sys
import os
import glob
import numpy as np
import pandas as pd
from utils import *

In [3]:
import SETTING
paths = SETTING.PATH(osg=False)
print(paths)
LOCATION = paths.LOCATION
USER = paths.USER
ICDATA_DIR = paths.ICDATA_DIR
DATA_DIR = paths.DATA_DIR
ANA_DIR = paths.ANA_DIR


#### Working at GT ####

LOCATION: 	 GT
USER: 	 cchen641
ICDATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/analyses/online_GRECO/version-002-p04
DATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data
ANA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data/csky_output



In [4]:
### create arguments arrays ---> 02.txt
# 0: grb_name_idx_start
# 1: grb_name_idx_end
# 2: tw_in_second

# Check 1 out of 2

In [5]:
####### Check before running ########
script_path = '/scratch/cjchen/submit/02.sub'
## [::-1] reverse order
grb_name_idx_starts = np.r_[0:2051:30]
grb_name_idx_ends   = np.r_[30:2051:30, 2051]
tws_in_second       = [  10,     25,    50,   100,   250,   500][::-1] 
#######################################

In [6]:
contents = ""
for grb_name_idx_start, grb_name_idx_end in zip(grb_name_idx_starts, grb_name_idx_ends):       
    for tw_in_second in tws_in_second:
        contents += "{}\t".format(grb_name_idx_start)
        contents += "{}\t".format(grb_name_idx_end)
        contents += "{}\n".format(tw_in_second)

with open("02.txt", 'w') as writer:
    writer.write(contents)
    

In [8]:
# print(contents)

In [9]:
print("Total number of jobs: {}\n".format(contents.count('\n')))

Total number of jobs: 414



In [10]:
### create pbs file ---> 02.pbs

# Check 2 out of 2

In [11]:
nameofjob  = "02"
ntotal     = 414             # copy from the above 2nd cell "Total number of jobs: xxx"
inputtxt   = "02.txt"      # copy from the above 3rd cell
outdir     = ensure_dir("/storage/home/hhive1/cchen641/scratch/icecube/job_pbs_out/out")
walltimehr = 120           # in Hour: 1 cpu min / trial (x2 for redundancy) < 120 hrs

In [12]:
contents = ""
contents += "#PBS -N {}\n".format(nameofjob)
contents += "#PBS -l nodes=1:ppn={}\n".format(1)           # number of nodes and cores per node required
contents += "#PBS -l pmem=4gb\n"                             # memory per core
contents += "#PBS -l walltime={}:00:00\n".format(walltimehr)  # dd:hh:mm:ss
contents += "#PBS -q hive\n"
contents += "#PBS -j oe\n"
contents += "#PBS -o {}\n".format(outdir+"/02.out")
contents += "#PBS -t 1-{}%400".format(ntotal)
contents += "\n"
contents += "cd /storage/home/hhive1/cchen641/icecube/greco_grb/scripts\n"
contents += "icpy3\n" 
# Get Paramters from input.txt file using $PBS_ARRAYID as the line number 
contents += 'params=`sed -n "${{PBS_ARRAYID}} p" {}`\n'.format("hive_sub/"+inputtxt)    # outer single quote to preserve inner double quotes
contents += "paramsArray=($params)\n"
contents += "arg0=${paramsArray[0]}\n"
contents += "arg1=${paramsArray[1]}\n"
contents += "arg2=${paramsArray[2]}\n"
contents += "python 02_get_background_TS_with_healpix.py "

contents += "--grb_name_idx_start $arg0 "
contents += "--grb_name_idx_end $arg1 "
contents += "--tw_in_second $arg2 "
print(contents)
with open("02.pbs", 'w') as writer:
    writer.write(contents)

#PBS -N 02
#PBS -l nodes=1:ppn=1
#PBS -l pmem=4gb
#PBS -l walltime=120:00:00
#PBS -q hive
#PBS -j oe
#PBS -o /storage/home/hhive1/cchen641/scratch/icecube/job_pbs_out/out/02.out
#PBS -t 1-414%400
cd /storage/home/hhive1/cchen641/icecube/greco_grb/scripts
icpy3
params=`sed -n "${PBS_ARRAYID} p" hive_sub/02.txt`
paramsArray=($params)
arg0=${paramsArray[0]}
arg1=${paramsArray[1]}
arg2=${paramsArray[2]}
python 02_get_background_TS_with_healpix.py --grb_name_idx_start $arg0 --grb_name_idx_end $arg1 --tw_in_second $arg2 
